In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_path = '/content/drive/MyDrive/PlantVillage/PlantVillage/Pepper'

In [ ]:
import os
from os import listdir
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf
import os
import tensorflow.keras as keras
import re
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define the source directory containing your dataset
source_dir = "/content/drive/MyDrive/PlantVillage/PlantVillage/Pepper"

# Define the destination directories for training and validation data
train_dir = "pepper/train"
valid_dir = "pepper/valid"

# Create the destination directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

# Get a list of class folders in the source directory
class_folders = os.listdir(source_dir)

# Define the ratio of data to put in the validation set (e.g., 20%)
validation_ratio = 0.2

# Loop through each class folder and split the data
for class_folder in class_folders:
    class_path = os.path.join(source_dir, class_folder)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        # Split the images into training and validation sets
        train_images, valid_images = train_test_split(images, test_size=validation_ratio, random_state=42)

        # Create destination directories for this class in the training and validation sets
        train_class_dir = os.path.join(train_dir, class_folder)
        valid_class_dir = os.path.join(valid_dir, class_folder)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(valid_class_dir, exist_ok=True)

        # Copy training images
        for image in train_images:
            source_image_path = os.path.join(class_path, image)
            dest_image_path = os.path.join(train_class_dir, image)
            shutil.copy(source_image_path, dest_image_path)

        # Copy validation images
        for image in valid_images:
            source_image_path = os.path.join(class_path, image)
            dest_image_path = os.path.join(valid_class_dir, image)
            shutil.copy(source_image_path, dest_image_path)

print("Dataset split into training and validation sets.")

Dataset split into training and validation sets.


In [ ]:
TRAIN_DIR = "pepper/train"
VALID_DIR = "pepper/valid"
def count_files_in_directory(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files

num_train_files = count_files_in_directory(TRAIN_DIR)
num_valid_files = count_files_in_directory(VALID_DIR)

print("Number of files in the training directory:", num_train_files)
print("Number of files in the validation directory:", num_valid_files)


Number of files in the training directory: 1979
Number of files in the validation directory: 496


In [ ]:
TRAIN_DIR = "pepper/train"

def get_classes_and_count(directory):
    classes = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    return classes, len(classes)

class_names, CLASS_NUMS = get_classes_and_count(TRAIN_DIR)

print("Number of classes in the training directory:", CLASS_NUMS)
print("Class names in the training directory:")
for class_name in class_names:
    print(class_name)

Number of classes in the training directory: 2
Class names in the training directory:
Pepper__bell___healthy
Pepper__bell___Bacterial_spot


In [ ]:
import os
import shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# Define directories and constants
source_dir = "/content/drive/MyDrive/PlantVillage/PlantVillage/Pepper"
train_dir = "pepper/train"
valid_dir = "pepper/valid"
BATCH_SIZE = 32
IMAGE_SHAPE = (224, 224)
numEPOCHS = 7  # You can increase the number of epochs

# Data Augmentation
training_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

testing_aug = ImageDataGenerator(rescale=1./255.0)

# Load and preprocess data
training_data = training_aug.flow_from_directory(
    train_dir,
    batch_size=BATCH_SIZE,
    target_size=IMAGE_SHAPE,
    shuffle=True,
    class_mode='categorical'
)

testing_data = testing_aug.flow_from_directory(
    valid_dir,
    batch_size=BATCH_SIZE,
    target_size=IMAGE_SHAPE,
    class_mode='categorical'
)

# Define and compile the model
base_model = InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

model = keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(CLASS_NUMS, activation='softmax')
])

# Learning Rate Scheduler
def lr_scheduler(epoch):
    if epoch < 10:
        return 0.001
    else:
        return 0.0001

lr_schedule = LearningRateScheduler(lr_scheduler)

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Training with early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    training_data,
    validation_data=testing_data,
    epochs=numEPOCHS,
    steps_per_epoch=len(training_data),
    validation_steps=len(testing_data),
    callbacks=[early_stopping, lr_schedule]
)


Found 1979 images belonging to 2 classes.
Found 496 images belonging to 2 classes.
87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/7
62/62 [==============================] - 84s 594ms/step - loss: 0.2820 - accuracy: 0.9242 - val_loss: 158.1987 - val_accuracy: 0.4032 - lr: 0.0010
Epoch 2/7
62/62 [==============================] - 31s 498ms/step - loss: 0.1806 - accuracy: 0.9611 - val_loss: 23.4569 - val_accuracy: 0.4032 - lr: 0.0010
Epoch 3/7
62/62 [==============================] - 31s 498ms/step - loss: 0.1815 - accuracy: 0.9611 - val_loss: 5691.7417 - val_accuracy: 0.4677 - lr: 0.0010
Epoch 4/7
62/62 [==============================] - 31s 497ms/step - loss: 0.1759 - accuracy: 0.9555 - val_loss: 15559.1357 - val_accuracy: 0.4052 - lr: 0.0010
Epoch 5/7
62/62 [==============================] - 31s 501ms/step - loss: 0.2683 - accuracy: 0.9343 - val_loss: 40.5928 - val_accuracy: 0.6089 - lr: 0.0010
Epoch 6/7
62/62 [==============================] - 31s 501ms/step 

In [ ]:
model.save("pepper_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow import keras
import warnings

#loading the trained model
warnings.filterwarnings("ignore")
model = keras.models.load_model('pepper_model.h5')

#converting model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

#saving tflite model
with open('pepper.tflite', 'wb') as f:
    f.write(tflite_model)

#creating interpreter for our model
interpreter = tf.lite.Interpreter(model_path='pepper.tflite')
interpreter.allocate_tensors()

In [ ]:
from google.colab import files
files.download("pepper_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("pepper.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>